# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules:

In [ ]:
import gsd.hoomd
import numpy as np

from morphct.chromophores import amber_dict, get_chromo_ids_smiles
from morphct.system import System

In the cell below, we'll create a system object for morphct. This the main class that will hold all the information for our simulation. We'll need to give it a gsd file, path to an output directory, the frame of the gsd file to use, the scaling factor to convert the lengths in the gsd to Angstroms, and a dictionary to map particle types to elements. Here's our starting structure, an atomistic (not coarse-grain or united atom) gsd file with 2 p3ht 16-mers: 

In [ ]:
gsdfile = "p3ht-trajectory.gsd"

system = System(gsdfile, "output_p3ht", frame=-1, scale=3.5636, conversion_dict=amber_dict)
system.visualize_system()

Next let's use SMARTS matching to detect our chromophores. This SMARTS string shown below is for p3ht. 

Note: The positions/orientations in the final frame of the gsd file are not optimal, so we are using the first frame (before any distortion) for SMARTS matching and then mapping those indices to the final structure. Sometimes SMARTS matching is not the best method to find the chromophores--check the ITIC example to see another method.

In [ ]:
smarts_str = "c1cscc1CCCCCC"

with gsd.hoomd.open(gsdfile) as f:
    snap0 = f[0]

aaids = get_chromo_ids_smiles(snap0, smarts_str, system.conversion_dict)

Next let's add these chromophores to the system and visualize them. We only have donor species in this system, and they'll be colored purple.

In [ ]:
system.add_chromophores(aaids, "donor")

system.visualize_chromophores()

Next let's compute the energies required to run the KMC simulation. First, the neighbors will be calculated (using voronoi analysis) and then the single and dimer energies will be calculated and saved to a file. (So that the simulation can be restarted from this point more easily.)

In [ ]:
system.compute_energies()

We can check that the pair and singles inputs look reasonable. There won't be any bonds and hydrogen atoms should've been added.

In [ ]:
i = 86 # try any number from 0 to 191
print(f"Pair #{i}:")
system.visualize_qcc_input(i, single=False)

i = 0 # try any number from 0 to 31
print(f"Single #{i}:")
system.visualize_qcc_input(i)

Once the energy files are finished, we can use them to set the energy values of the chromophores . (This can also be run to restart the calculation from this point.)

In [ ]:
system.set_energies("output_p3ht/singles_energies.txt", "output_p3ht/dimer_energies.txt")

This function sets the `homo_1`, `homo`, `lumo`, `lumo_1`, `neighbors_delta_e`, and `neighbors_ti` attributes of each chromphore:

In [ ]:
i = 0
chromo = system.chromophores[i]
print(f"Chromophore {i}:")
print(
    f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} "
    f"LUMO+1: {chromo.lumo_1:.2f}"
)
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")

With all the energy values set, we're ready to run KMC! We need to set the temperature that the KMC simulation will be run at and the lifetimes and numbers of our carriers:

In [ ]:
lifetimes = [1.00e-13, 1.00e-12]
temp = 300
system.run_kmc(lifetimes, temp, n_holes=10, verbose=1)

The output files for each process are saved in `output_p3ht/kmc/kmc_PROC#.log` (where `PROC#` is whatever process number the job was run on) and analysis plots will be saved in `output_p3ht/kmc/figures/`.

In [ ]:
with open("output/kmc/kmc_00.log", "r") as f:
    lines = f.readlines()
print(*lines)